## Hill Climbing Algorithm in Python

In [2]:
import random

def randomSolution(tsp):
    """
    Since every city can only be visited one, after its identifier is added to
    our solution we then remove that city’s identifier from the city identifier list.
    Our function needs the Travelling salesman problem itself for information
    about the distances between cities
    """
    cities = list(range(len(tsp)))
    solution = []

    for i in range(len(tsp)):
        randomCity = cities[random.randint(0, len(cities) - 1)]
        solution.append(randomCity)
        cities.remove(randomCity)

    return solution

def routeLength(tsp, solution):
    """
    Since a solution is a list of all cities in a specific order, we can just iterate over a solution and
    use the tsp argument to add the distance to each new city to our total route length.
    The iterator i “visits” each city, so i-1 is “at” the previous city or
    the last city when i equals 0 (which is exactly what we want, since we want to end up at the first city again).
    solution[i] thus gives us the current city, and solution[i-1] gives us the previous one.
    Then we simply use the tsp to get the distance between these cities, which we add to the total length of the route (routeLength).
    """
    routeLength = 0
    for i in range(len(solution)):
        routeLength += tsp[solution[i - 1]][solution[i]]
    return routeLength

def getNeighbours(solution):
    """
    A neighbouring solution is a solution that’s only slightly different from the current solution. Note also that a neighbour still needs to
    be a correct solution: every city still needs to be visited exactly once. We can accomplish both by creating a neighbour as follows:
    copy the current solution, and then make two cities swap places! This way, we create a slightly different solution that’s still correct.
    Since we want to create all neighbours to a solution, and need to make cities swap places, we need to create two for loops, one nested in
    the other, both iterating the current solution. Since swapping city A with city B is the same as swapping
    city B with city A, our second loop needs to only loop over those cities the first loop hasn’t looped over yet. Inside the second loop,
    we create our neighbour with the swapped cities and add it to our neighbours list.
    """
    neighbours = []
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            neighbour = solution.copy()
            neighbour[i] = solution[j]
            neighbour[j] = solution[i]
            neighbours.append(neighbour)
    return neighbours

def getBestNeighbour(tsp, neighbours):
    """
     This is a pretty straightforward function: it first sets the bestNeighbour to the first neighbour in the list of
     neighbours (and bestRouteLength to its route length), and then iterates over all neighbours; when a neighbour has a
     shorter route length, both bestNeighbour and bestRouteLength are updated.
    """
    bestRouteLength = routeLength(tsp, neighbours[0])
    bestNeighbour = neighbours[0]
    for neighbour in neighbours:
        currentRouteLength = routeLength(tsp, neighbour)
        if currentRouteLength < bestRouteLength:
            bestRouteLength = currentRouteLength
            bestNeighbour = neighbour
    return bestNeighbour, bestRouteLength

def hillClimbing(tsp):

    currentSolution = randomSolution(tsp)
    currentRouteLength = routeLength(tsp, currentSolution)
    neighbours = getNeighbours(currentSolution)
    bestNeighbour, bestNeighbourRouteLength = getBestNeighbour(tsp, neighbours)

    while bestNeighbourRouteLength < currentRouteLength:
        currentSolution = bestNeighbour
        currentRouteLength = bestNeighbourRouteLength
        neighbours = getNeighbours(currentSolution)
        bestNeighbour, bestNeighbourRouteLength = getBestNeighbour(tsp, neighbours)

    return currentSolution, currentRouteLength

def main():
    tsp = [
        [0, 400, 500, 300],
        [400, 0, 300, 500],
        [500, 300, 0, 400],
        [300, 500, 400, 0]
    ]

    print(hillClimbing(tsp))

main()

([1, 2, 3, 0], 1400)


## Simulated Annealing in Python

In [4]:
"""
make a random initial guess
set initial temperature
loop many times
  swap two randomly selected elements of the guess
  compute error of proposed solution
  if proposed solution is better than curr solution then
    accept the proposed solution
  else if proposed solution is worse then
    accept the worse solution anyway with small probability
  else
    don't accept the proposed solution
  end-if
  reduce temperature slightly
end-loop
return best solution found
"""

"\nmake a random initial guess\nset initial temperature\nloop many times\n  swap two randomly selected elements of the guess\n  compute error of proposed solution\n  if proposed solution is better than curr solution then\n    accept the proposed solution\n  else if proposed solution is worse then\n    accept the worse solution anyway with small probability\n  else\n    don't accept the proposed solution\n  end-if\n  reduce temperature slightly\nend-loop\nreturn best solution found\n"

In [5]:
import numpy as np

def total_dist(route):
  d = 0.0  # total distance between cities
  n = len(route)
  for i in range(n-1):
    if route[i] < route[i+1]:
      d += (route[i+1] - route[i]) * 1.0
    else:
      d += (route[i] - route[i+1]) * 1.5
  return d

def error(route):
  n = len(route)
  d = total_dist(route)
  min_dist = n-1
  return d - min_dist

def adjacent(route, rnd):
  n = len(route)
  result = np.copy(route)
  i = rnd.randint(n); j = rnd.randint(n)
  tmp = result[i]
  result[i] = result[j]; result[j] = tmp
  return result

def solve(n_cities, rnd, max_iter,start_temperature, alpha):
  # solve using simulated annealing
  curr_temperature = start_temperature
  soln = np.arange(n_cities, dtype=np.int64)
  rnd.shuffle(soln)
  print("Initial guess: ")
  print(soln)

  err = error(soln)
  iteration = 0
  interval = (int)(max_iter / 10)
  while iteration < max_iter and err > 0.0:
    adj_route = adjacent(soln, rnd)
    adj_err = error(adj_route)

    if adj_err < err:  # better route so accept
      soln = adj_route; err = adj_err
    else:          # adjacent is worse
      accept_p = \
        np.exp((err - adj_err) / curr_temperature)
      p = rnd.random()
      if p < accept_p:  # accept anyway
        soln = adj_route; err = adj_err
      # else don't accept

    if iteration % interval == 0:
      print("iter = %6d | curr error = %7.2f | \
      temperature = %10.4f " % \
      (iteration, err, curr_temperature))

    if curr_temperature < 0.00001:
      curr_temperature = 0.00001
    else:
      curr_temperature *= alpha
    iteration += 1

  return soln

def main():
  print("\nBegin TSP simulated annealing demo ")

  num_cities = 20
  print("\nSetting num_cities = %d " % num_cities)
  print("\nOptimal solution is 0, 1, 2, . . " + \
    str(num_cities-1))
  print("Optimal solution has total distance = %0.1f " \
    % (num_cities-1))
  rnd = np.random.RandomState(4)
  max_iter = 2500
  start_temperature = 10000.0
  alpha = 0.99

  print("\nSettings: ")
  print("max_iter = %d " % max_iter)
  print("start_temperature = %0.1f " \
    % start_temperature)
  print("alpha = %0.2f " % alpha)

  print("\nStarting solve() ")
  soln = solve(num_cities, rnd, max_iter,
    start_temperature, alpha)
  print("Finished solve() ")

  print("\nBest solution found: ")
  print(soln)
  dist = total_dist(soln)
  print("\nTotal distance = %0.1f " % dist)

  print("\nEnd demo ")

if __name__ == "__main__":
  main()


Begin TSP simulated annealing demo 

Setting num_cities = 20 

Optimal solution is 0, 1, 2, . . 19
Optimal solution has total distance = 19.0 

Settings: 
max_iter = 2500 
start_temperature = 10000.0 
alpha = 0.99 

Starting solve() 
Initial guess: 
[19  3 18  6 13  4  0 17 12 11 15 10  9  2 16  7  8  1  5 14]
iter =      0 | curr error =  148.50 |       temperature = 10000.0000 
iter =    250 | curr error =  119.50 |       temperature =   810.5852 
iter =    500 | curr error =  137.50 |       temperature =    65.7048 
iter =    750 | curr error =   73.00 |       temperature =     5.3259 
iter =   1000 | curr error =   32.50 |       temperature =     0.4317 
iter =   1250 | curr error =   25.00 |       temperature =     0.0350 
iter =   1500 | curr error =   20.00 |       temperature =     0.0028 
iter =   1750 | curr error =   17.50 |       temperature =     0.0002 
iter =   2000 | curr error =   12.50 |       temperature =     0.0000 
iter =   2250 | curr error =    5.00 |       tem

## Genetic Algorithm

In [6]:
def is_safe(board, row, col, N):
    # Check if there is a queen in the same column
    for i in range(row):
        if board[i][col] == 1:
            return False

    # Check upper left diagonal
    for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False

    # Check upper right diagonal
    for i, j in zip(range(row, -1, -1), range(col, N)):
        if board[i][j] == 1:
            return False

    return True

def solve_n_queens_util(board, row, N, solutions):
    if row == N:
        solutions.append([list(row) for row in board])
        return

    for col in range(N):
        if is_safe(board, row, col, N):
            board[row][col] = 1
            solve_n_queens_util(board, row + 1, N, solutions)
            board[row][col] = 0

def solve_n_queens(N):
    board = [[0 for _ in range(N)] for _ in range(N)]
    solutions = []

    solve_n_queens_util(board, 0, N, solutions)

    return solutions

def print_board(board):
    for row in board:
        print(' '.join('Q' if cell == 1 else '.' for cell in row))

# Example usage:
N = 8
solutions = solve_n_queens(N)

for i, solution in enumerate(solutions):
    print(f"Solution {i + 1}:")
    print_board(solution)
    print()

Solution 1:
Q . . . . . . .
. . . . Q . . .
. . . . . . . Q
. . . . . Q . .
. . Q . . . . .
. . . . . . Q .
. Q . . . . . .
. . . Q . . . .

Solution 2:
Q . . . . . . .
. . . . . Q . .
. . . . . . . Q
. . Q . . . . .
. . . . . . Q .
. . . Q . . . .
. Q . . . . . .
. . . . Q . . .

Solution 3:
Q . . . . . . .
. . . . . . Q .
. . . Q . . . .
. . . . . Q . .
. . . . . . . Q
. Q . . . . . .
. . . . Q . . .
. . Q . . . . .

Solution 4:
Q . . . . . . .
. . . . . . Q .
. . . . Q . . .
. . . . . . . Q
. Q . . . . . .
. . . Q . . . .
. . . . . Q . .
. . Q . . . . .

Solution 5:
. Q . . . . . .
. . . Q . . . .
. . . . . Q . .
. . . . . . . Q
. . Q . . . . .
Q . . . . . . .
. . . . . . Q .
. . . . Q . . .

Solution 6:
. Q . . . . . .
. . . . Q . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . . . . Q
. . . . . Q . .
. . . Q . . . .

Solution 7:
. Q . . . . . .
. . . . Q . . .
. . . . . . Q .
. . . Q . . . .
Q . . . . . . .
. . . . . . . Q
. . . . . Q . .
. . Q . . . . .

Solution 8:
.

In [ ]:
# N queen problem
import random

def random_chromosome(size): #making random chromosomes
    return [ random.randint(1, nq) for _ in range(nq) ]

def fitness(chromosome):
    horizontal_collisions = sum([chromosome.count(queen)-1 for queen in chromosome])/2
    diagonal_collisions = 0

    n = len(chromosome)
    left_diagonal = [0] * 2*n
    right_diagonal = [0] * 2*n
    for i in range(n):
        left_diagonal[i + chromosome[i] - 1] += 1
        right_diagonal[len(chromosome) - i + chromosome[i] - 2] += 1

    diagonal_collisions = 0
    for i in range(2*n-1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i]-1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i]-1
        diagonal_collisions += counter / (n-abs(i-n+1))

    return int(maxFitness - (horizontal_collisions + diagonal_collisions)) #28-(2+3)=23

def probability(chromosome, fitness):
    return fitness(chromosome) / maxFitness

def random_pick(population, probabilities):
    populationWithProbabilty = zip(population, probabilities)
    total = sum(w for c, w in populationWithProbabilty)
    r = random.uniform(0, total)
    upto = 0
    for c, w in zip(population, probabilities):
        if upto + w >= r:
            return c
        upto += w
    assert False, "Shouldn't get here"

def reproduce(x, y): #doing cross_over between two chromosomes
    n = len(x)
    c = random.randint(0, n - 1)
    return x[0:c] + y[c:n]

def mutate(x):  #randomly changing the value of a random index of a chromosome
    n = len(x)
    c = random.randint(0, n - 1)
    m = random.randint(1, n)
    x[c] = m
    return x

def genetic_queen(population, fitness):
    mutation_probability = 0.03
    new_population = []
    probabilities = [probability(n, fitness) for n in population]
    for i in range(len(population)):
        x = random_pick(population, probabilities) #best chromosome 1
        y = random_pick(population, probabilities) #best chromosome 2
        child = reproduce(x, y) #creating two new chromosomes from the best 2 chromosomes
        if random.random() < mutation_probability:
            child = mutate(child)
        print_chromosome(child)
        new_population.append(child)
        if fitness(child) == maxFitness: break
    return new_population

def print_chromosome(chrom):
    print("Chromosome = {},  Fitness = {}"
        .format(str(chrom), fitness(chrom)))

if __name__ == "__main__":
    nq = 8#int(input("Enter Number of Queens: ")) #say N = 8
    maxFitness = (nq*(nq-1))/2  # 8*7/2 = 28
    population = [random_chromosome(nq) for _ in range(100)]

    generation = 1

    while not maxFitness in [fitness(chrom) for chrom in population]:
        print("=== Generation {} ===".format(generation))
        population = genetic_queen(population, fitness)
        print("")
        print("Maximum Fitness = {}".format(max([fitness(n) for n in population])))
        generation += 1
    chrom_out = []
    print("Solved in Generation {}!".format(generation-1))
    for chrom in population:
        if fitness(chrom) == maxFitness:
            print("");
            print("One of the solutions: ")
            chrom_out = chrom
            print_chromosome(chrom)

    board = []

    for x in range(nq):
        board.append(["x"] * nq)

    for i in range(nq):
        board[nq-chrom_out[i]][i]="Q"


    def print_board(board):
        for row in board:
            print (" ".join(row))

    print()
    print_board(board)

Streaming output truncated to the last 5000 lines.
Chromosome = [4, 2, 1, 4, 6, 7, 8, 3],  Fitness = 26
Chromosome = [4, 7, 1, 5, 6, 7, 8, 3],  Fitness = 26
Chromosome = [4, 2, 1, 5, 6, 7, 8, 3],  Fitness = 27
Chromosome = [4, 2, 1, 5, 6, 7, 8, 6],  Fitness = 26
Chromosome = [4, 2, 1, 5, 6, 7, 8, 3],  Fitness = 27
Chromosome = [4, 2, 7, 5, 6, 7, 8, 3],  Fitness = 26
Chromosome = [4, 7, 1, 5, 6, 7, 3, 3],  Fitness = 25
Chromosome = [4, 2, 1, 5, 6, 7, 8, 3],  Fitness = 27
Chromosome = [4, 2, 7, 5, 6, 7, 8, 3],  Fitness = 26
Chromosome = [4, 4, 1, 5, 5, 7, 8, 3],  Fitness = 25
Chromosome = [4, 2, 1, 5, 6, 7, 8, 6],  Fitness = 26
Chromosome = [4, 2, 7, 5, 6, 7, 8, 3],  Fitness = 26
Chromosome = [4, 2, 1, 5, 5, 7, 8, 3],  Fitness = 26
Chromosome = [4, 2, 7, 5, 6, 7, 3, 3],  Fitness = 25
Chromosome = [8, 2, 1, 5, 6, 7, 8, 3],  Fitness = 25
Chromosome = [4, 2, 1, 5, 6, 7, 8, 3],  Fitness = 27
Chromosome = [4, 2, 1, 5, 6, 7, 8, 3],  Fitness = 27
Chromosome = [4, 2, 1, 5, 6, 7, 8, 3],  Fitness 

KeyboardInterrupt: ignored

In [ ]:
import random

def fitness(perm):
    # Calculate the number of non-attacking pairs of queens (i.e., number of conflicts)
    N = len(perm)
    conflicts = 0
    for i in range(N):
        for j in range(i + 1, N):
            if perm[i] == perm[j] or abs(perm[i] - perm[j]) == abs(i - j):
                conflicts += 1
    return N * (N - 1) // 2 - conflicts

def generate_random_permutation(N):
    return random.sample(range(N), N)

def crossover(parent1, parent2):
    N = len(parent1)
    crossover_point = random.randint(1, N - 1)
    child1 = parent1[:crossover_point] + [x for x in parent2 if x not in parent1[:crossover_point]]
    child2 = parent2[:crossover_point] + [x for x in parent1 if x not in parent2[:crossover_point]]
    return child1, child2

def mutate(perm, mutation_rate):
    N = len(perm)
    for i in range(N):
        if random.random() < mutation_rate:
            j = random.randint(0, N - 1)
            perm[i], perm[j] = perm[j], perm[i]
    return perm

def genetic_algorithm(N, population_size, mutation_rate, max_generations):
    population = [generate_random_permutation(N) for _ in range(population_size)]

    for generation in range(max_generations):
        fitness_scores = [fitness(perm) for perm in population]
        best_perm = population[fitness_scores.index(max(fitness_scores))]

        if max(fitness_scores) == N * (N - 1) // 2:
            return best_perm

        mating_pool = random.choices(population, weights=fitness_scores, k=population_size)
        new_population = []

        for _ in range(population_size // 2):
            parent1, parent2 = random.sample(mating_pool, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population

    return best_perm

def print_board(perm):
    N = len(perm)
    for row in range(N):
        line = ['.'] * N
        line[perm[row]] = 'Q'
        print(' '.join(line))

# Example usage:
N = 8
population_size = 100
mutation_rate = 0.05
max_generations = 1000

solution = genetic_algorithm(N, population_size, mutation_rate, max_generations)
print_board(solution)


. . Q . . . . .
. . . . . . Q .
. Q . . . . . .
. . . . . . . Q
. . . . . Q . .
. . . Q . . . .
Q . . . . . . .
. . . . Q . . .
